In [3]:
#import matplotlib
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [4]:
housing_df = pd.read_csv("Resources\DC_Properties.csv")

C:\Users\djhon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
housing_df['YEAR'] = pd.DatetimeIndex(housing_df['SALEDATE']).year

In [ ]:
housing_df.columns

In [6]:
#Filter orginal dataframe to get to meaningful data 54,752 rows
hdf2 = housing_df.loc[housing_df['YEAR']== 2017, :]
hdf3 = hdf2.loc[hdf2['SOURCE'] == 'Residential', : ]
hdf4 = hdf3.dropna(subset=['PRICE','QUADRANT','WARD','FULLADDRESS','ZIPCODE','BEDRM','BATHRM','KITCHENS'])
hdf5 = hdf4[['QUADRANT','FULLADDRESS','ZIPCODE','BEDRM','BATHRM','KITCHENS','PRICE']]
hdf6 = hdf5.loc[hdf5['PRICE'] > 100000, :]
hdtable = pd.DataFrame(hdf6)

In [ ]:
hdtable.head(5)

In [ ]:
hdtable.columns

In [ ]:
hdtable.info()

In [ ]:
hdtable.count()

In [ ]:
grouped_year = hdtable.groupby("QUADRANT").PRICE.agg(["min", "mean","median", "max", "count"])
grouped_year

In [7]:
hdtable.to_csv("hdtable.csv", index=False, header=True)

In [8]:
test=pd.read_csv("hdtable.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237 entries, 0 to 5236
Data columns (total 7 columns):
QUADRANT       5237 non-null object
FULLADDRESS    5237 non-null object
ZIPCODE        5237 non-null float64
BEDRM          5237 non-null int64
BATHRM         5237 non-null int64
KITCHENS       5237 non-null float64
PRICE          5237 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 286.5+ KB


In [9]:
# Create Engine
engine = create_engine("sqlite:///housingdata.sqlite")
# Declare a Base
Base = declarative_base()

In [10]:
# Define the ORM class for `Measurements`
class Measurement(Base):
    
    __tablename__ = 'hdata'

    id = Column(Integer, primary_key=True)
    QUADRANT = Column(String)
    FULLADDRESS = Column(String)
    ZIPCODE = Column(Integer)
    BEDRM = Column(Integer)
    BATHRM = Column(Integer)
    KITCHENS = Column(Integer)
    PRICE = Column(Float)

In [11]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)

In [12]:
# Verify that the table names exist in the database
engine.table_names()

['hdata']

In [20]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

In [21]:
populate_table(engine, Measurement.__table__, 'hdtable.csv')

In [22]:
engine.execute("SELECT * FROM hdata LIMIT 10").fetchall()

[(1, 'NW', '1416 CORCORAN STREET NW', 20009, 4, 3, 2, 1700000.0),
 (2, 'NW', '1910 1/2 NEW HAMPSHIRE AVENUE NW', 20009, 3, 2, 2, 1363000.0),
 (3, 'NW', '1619 SWANN STREET NW', 20009, 4, 6, 4, 1500000.0),
 (4, 'NW', '1707 21ST STREET NW', 20009, 6, 4, 4, 1351830.0),
 (5, 'NW', '2025 R STREET NW', 20009, 6, 4, 2, 1844970.0),
 (6, 'NW', '2014 HILLYER PLACE NW', 20009, 5, 3, 2, 1625000.0),
 (7, 'NW', '1760 T STREET NW', 20009, 5, 2, 2, 1925000.0),
 (8, 'NW', '1808 CORCORAN STREET NW', 20009, 3, 2, 1, 950000.0),
 (9, 'NW', '1639 19TH STREET NW', 20009, 3, 2, 2, 1330000.0),
 (10, 'NW', '1524 18TH STREET NW', 20036, 4, 4, 4, 2279000.0)]